In [1]:
%matplotlib notebook
import warnings
warnings.filterwarnings('ignore')
import numpy as np
from scipy.ndimage import convolve
import matplotlib
matplotlib.use('WebAgg')
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib import style
style.use('dark_background')
from IPython.display import HTML

In [26]:
constant = 42 # width and length of the grid
probability = 0.5 # distribution of ones on the grid
size = (constant, constant) 
# matplotlib animation variables
frames = 120  
interval = 100

In [25]:
# initialize the grids
status = np.random.binomial(1, probability, size=size) # initialize grid
base = np.zeros(size) # for the tracking purpose

# styling the plot
matplotlib.rc('axes', edgecolor='blue')
fig, ax = plt.subplots(1, figsize=(6,6))
ticks = np.arange(-.5, constant, 1)
ax.set_xticks(ticks)
ax.set_yticks(ticks)
plt.tick_params(
    axis='both',      
    which='both', 
    color='blue',    
    bottom=True,      
    top=True,  
    left=True,
    right=True,        
    labelbottom=False,
    labelleft=False)
ax.set_xlim(-.1,constant-.1)
ax.set_ylim(-.1, constant-.1)
ax.grid(c='k',lw=2, alpha=.75)


def check_alive(status):
    '''
    Counting neighbors using the convolution opearation.
    '''
    kernel = np.array([[1, 1, 1],
                       [1, 0, 1],
                       [1, 1, 1]])
    convolv = convolve(status, kernel, mode='constant', cval=0.)
    return convolv
 
def trace(status):
    """records the history of the living cells on the grid"""
    global base
    
    base += status * .01 
    base -= .001
    
    return np.clip(base, 0, 1) 
 
def next_gen(status):
    '''implementing the rules'''
    # Ah, ha, ha, ha
    stayin_alive = check_alive(status)
    two_neigh = np.array((stayin_alive == 2)) * status # multiplying with status prevents the newborn cell 
    three_neigh = np.array((stayin_alive == 3))
    new_status = (two_neigh + three_neigh)
 
    return new_status.astype(np.uint8)
 
im = plt.imshow(next_gen(status), animated=True, cmap='gnuplot2')
 
def update(frame):
    global status
    status = next_gen(status)
    tracer = trace(status)
    result = (status + tracer).clip(0,1)
    im.set_array(result)
    return im, 
 
ani = animation.FuncAnimation(fig, update, interval=interval, 
                              frames=frames, blit=True)  
 
HTML(ani.to_html5_video())